---
## OpenSearch 起動・停止

* 起動

In [1]:
%%bash
docker compose -f "../0_opensearch-docker/docker-compose.yml" up -d


 Container opensearch-dashboards  Created
 Container opensearch-node1  Created
 Container opensearch-dashboards  Starting
 Container opensearch-node1  Starting
 Container opensearch-dashboards  Started
 Container opensearch-node1  Started


* 停止

In [53]:
%%bash
docker compose -f "../0_opensearch-docker/docker-compose.yml" down
# docker compose -f "../0_opensearch-docker/docker-compose.yml" down -v


 Container opensearch-node1  Stopping
 Container opensearch-dashboards  Stopping
 Container opensearch-dashboards  Stopped
 Container opensearch-dashboards  Removing
 Container opensearch-dashboards  Removed
 Container opensearch-node1  Stopped
 Container opensearch-node1  Removing
 Container opensearch-node1  Removed
 Network 0_opensearch-docker_opensearch-net  Removing
 Network 0_opensearch-docker_opensearch-net  Removed


In [2]:
%%bash
docker compose -f "../0_opensearch-docker/docker-compose.yml" logs


opensearch-node1  | Disabling execution of install_demo_configuration.sh for OpenSearch Security Plugin
opensearch-node1  | Disabling OpenSearch Security Plugin
opensearch-node1  | Enabling execution of OPENSEARCH_HOME/bin/opensearch-performance-analyzer/performance-analyzer-agent-cli for OpenSearch Performance Analyzer Plugin
opensearch-node1  | WARNING: A terminally deprecated method in java.lang.System has been called
opensearch-node1  | WARNING: System::setSecurityManager has been called by org.opensearch.bootstrap.OpenSearch (file:/usr/share/opensearch/lib/opensearch-2.11.0.jar)
opensearch-node1  | WARNING: Please consider reporting this to the maintainers of org.opensearch.bootstrap.OpenSearch
opensearch-node1  | WARNING: System::setSecurityManager will be removed in a future release
opensearch-node1  | WARNING: A terminally deprecated method in java.lang.System has been called
opensearch-node1  | WARNING: System::setSecurityManager has been called by org.opensearch.bootstrap.Sec

---
## 事前準備


In [26]:
%pip install -Uq opensearch-py langchain beautifulsoup4 requests boto3 python-dotenv



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
import os
import requests


In [4]:
from dotenv import load_dotenv

load_dotenv(override=True)


True

In [5]:
host = 'localhost' #'172.17.0.1'
port = 9200


In [6]:
from opensearchpy import OpenSearch

client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    use_ssl = False,
    verify_certs = False
)

info = client.info()
print(f"Welcome to {info['version']['distribution']} {info['version']['number']}!")


Welcome to opensearch 2.11.0!


---
## 1. Amazon BedrockとのMLコネクターを作成


In [8]:
body = {
  "name": "Amazon Bedrock Connector: embedding",
  "description": "The connector to the Bedrock Titan embedding model",
  "version": 1,
  "protocol": "aws_sigv4",
  "parameters": {
    "region": os.getenv("AWS_DEFAULT_REGION"),
    "service_name": "bedrock"
  },
  "credential": {
    "access_key": os.getenv("AWS_ACCESS_KEY_ID"),
      "secret_key": os.getenv("AWS_SECRET_ACCESS_KEY")
  },
  "actions": [
    {
      "action_type": "predict",
      "method": "POST",
      "url": "https://bedrock-runtime.us-east-1.amazonaws.com/model/amazon.titan-embed-text-v1/invoke",
      "headers": {
        "content-type": "application/json",
        "x-amz-content-sha256": "required"
      },
      "request_body": "{ \"inputText\": \"${parameters.inputText}\" }",
      "pre_process_function": "\n    StringBuilder builder = new StringBuilder();\n    builder.append(\"\\\"\");\n    String first = params.text_docs[0];\n    builder.append(first);\n    builder.append(\"\\\"\");\n    def parameters = \"{\" +\"\\\"inputText\\\":\" + builder + \"}\";\n    return  \"{\" +\"\\\"parameters\\\":\" + parameters + \"}\";",
      "post_process_function": "\n      def name = \"sentence_embedding\";\n      def dataType = \"FLOAT32\";\n      if (params.embedding == null || params.embedding.length == 0) {\n        return params.message;\n      }\n      def shape = [params.embedding.length];\n      def json = \"{\" +\n                 \"\\\"name\\\":\\\"\" + name + \"\\\",\" +\n                 \"\\\"data_type\\\":\\\"\" + dataType + \"\\\",\" +\n                 \"\\\"shape\\\":\" + shape + \",\" +\n                 \"\\\"data\\\":\" + params.embedding +\n                 \"}\";\n      return json;\n    "
    }
  ]
}

response = requests.post(
  f'http://{host}:{port}/_plugins/_ml/connectors/_create', 
  json=body
  )

response.json()


{'connector_id': 'P1KL2IsBoHnPTWrQwH_H'}

In [9]:
connector_id = response.json()["connector_id"]
connector_id


'P1KL2IsBoHnPTWrQwH_H'

---
## 2. モデルグループを作成


In [10]:
model_group_name = "bedrock-embedded-model-group"

body = {
    "name": model_group_name
}

response = requests.post(
  f'http://{host}:{port}/_plugins/_ml/model_groups/_register', 
  json=body
  )

response.json()


{'model_group_id': 'QFKM2IsBoHnPTWrQC38q', 'status': 'CREATED'}

In [11]:
model_group_id = response.json()["model_group_id"]
model_group_id


'QFKM2IsBoHnPTWrQC38q'

---
## 3. モデルを作成


In [12]:
model_name = "bedrock-embedded-model"

body = {
    "name": model_name,
    "function_name": "remote",
    "model_group_id": model_group_id,
    "connector_id": connector_id
}

response = requests.post(
  f'http://{host}:{port}/_plugins/_ml/models/_register', 
  json=body
  )

response.json()


{'task_id': 'QVKM2IsBoHnPTWrQWX9z',
 'status': 'CREATED',
 'model_id': 'QlKM2IsBoHnPTWrQXn_-'}

In [13]:
model_id = response.json()["model_id"]
model_id


'QlKM2IsBoHnPTWrQXn_-'

---
## 4. モデルをデプロイ


In [14]:
response = requests.post(
  f'http://{host}:{port}/_plugins/_ml/models/{model_id}/_deploy', 
  )

response.json()


{'task_id': 'Q1KM2IsBoHnPTWrQlX8y',
 'task_type': 'DEPLOY_MODEL',
 'status': 'COMPLETED'}

---
## 5. Ingestパイプラインを作成


In [15]:
ingest_pipeline_name = "nlp-ingest-pipeline"
embedding_field = "passage_embedding"
embedding_target_field = "question"

body = {
  "description": "An NLP ingest pipeline",
  "processors": [
    {
      "text_embedding": {
        "model_id": model_id,
        "field_map": {
          embedding_target_field: embedding_field
        }
      }
    }
  ]
}

response = requests.put(
  f'http://{host}:{port}/_ingest/pipeline/{ingest_pipeline_name}', 
  json=body
  )

response.json()


{'acknowledged': True}

---
## 6. Searchパイプラインを作成


In [16]:
search_pipeline_name = 'nlp-search-pipeline'

body = {
  "description": "Post processor for hybrid search",
  "phase_results_processors": [
    {
      "normalization-processor": {
        "normalization": {
          "technique": "min_max"
        },
        "combination": {
          "technique": "arithmetic_mean",
          "parameters": {
            "weights": [
              0.3,
              0.7
            ]
          }
        }
      }
    }
  ]
}

response = requests.put(
  f'http://{host}:{port}/_search/pipeline/{search_pipeline_name}', 
  json=body
  )

response.json()


{'acknowledged': True}

---
## 1. インデックスを作成


In [17]:
index_name = 'semantic-index'


In [18]:
body = {
  "settings": {
    "index.knn": True,
    "default_pipeline": ingest_pipeline_name,
    "index": {
      "analysis": {
        "analyzer": {
          "custom_kuromoji_analyzer": {
            "tokenizer": "kuromoji_tokenizer",
            "filter": ["kuromoji_baseform", "ja_stop"],
            "char_filter": ["icu_normalizer"]
          }
        }
      }
    }
  },
  "mappings": {
    "properties": {
      embedding_field: {
        "type": "knn_vector",
        "dimension": 1536
      },
      "question": {"type": "text", "analyzer": "custom_kuromoji_analyzer"},
      "answer": {"type": "text", "analyzer": "custom_kuromoji_analyzer"}
    }
  }
}

response = client.indices.create(
  index_name, 
  body=body
)

print(response)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'semantic-index'}


---
## 2. データを登録


In [19]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter

loader = WebBaseLoader("https://aws.amazon.com/jp/ec2/faqs/")
data = loader.load()

text_splitter = CharacterTextSplitter(
    separator = "Q:",
    keep_separator=True,
    chunk_size = 10,
    chunk_overlap  = 0,
)

# 先頭のいらないものを消す
texts = text_splitter.split_documents(data)
texts = texts[1:]
# 末尾のいらないものを消す
texts[-1].page_content = texts[-1].page_content.split('\xa0')[0]


Created a chunk of size 1283, which is longer than the specified 10
Created a chunk of size 154, which is longer than the specified 10
Created a chunk of size 425, which is longer than the specified 10
Created a chunk of size 275, which is longer than the specified 10
Created a chunk of size 190, which is longer than the specified 10
Created a chunk of size 582, which is longer than the specified 10
Created a chunk of size 832, which is longer than the specified 10
Created a chunk of size 483, which is longer than the specified 10
Created a chunk of size 203, which is longer than the specified 10
Created a chunk of size 826, which is longer than the specified 10
Created a chunk of size 211, which is longer than the specified 10
Created a chunk of size 574, which is longer than the specified 10
Created a chunk of size 322, which is longer than the specified 10
Created a chunk of size 199, which is longer than the specified 10
Created a chunk of size 365, which is longer than the specifi

In [20]:
for text in texts:
  try:
    lines = text.page_content.splitlines()
    q = lines[0]
    a = text.page_content

    body = {
      "question": q,
      "answer": a
    }

    response = client.index(
      index=index_name,
      body=body
    )

    print(response)
  except Exception as e:
    print(e)


{'_index': 'semantic-index', '_id': 'RFKN2IsBoHnPTWrQ8H87', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}
{'_index': 'semantic-index', '_id': 'RVKN2IsBoHnPTWrQ838V', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1}
{'_index': 'semantic-index', '_id': 'RlKN2IsBoHnPTWrQ9n8O', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1}
{'_index': 'semantic-index', '_id': 'R1KN2IsBoHnPTWrQ-H-U', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 1}
{'_index': 'semantic-index', '_id': 'SFKN2IsBoHnPTWrQ-391', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 4, '_primary_term': 1}
{'_index': 'semantic-index', '_id': 'SVKN2IsBoHnPTWrQ_n9Q', '_version': 1, 

---
## 1. キーワード検索


In [21]:
question = "EC2で起動できるOSは何ですか？"


In [41]:
from langchain.chat_models import BedrockChat

llm = BedrockChat(model_id="anthropic.claude-instant-v1", model_kwargs={"max_tokens_to_sample": 4000})

In [28]:
from typing import Any, Iterable, List

from langchain.callbacks.manager import CallbackManagerForRetrieverRun
from langchain.docstore.document import Document
from langchain.schema import BaseRetriever

class OpenSearchBM25Retriever(BaseRetriever):

    client: Any
    index_name: str

    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:

        body = {
            "_source": {
                "excludes": [
                    "passage_embedding"
                ]
            },
            "query": {
                "match": {
                    "question": {
                        "query": query
                    }
                }
            }
        }

        res  = self.client.search(index=index_name,body=body)

        docs = []
        for r in res["hits"]["hits"]:
            docs.append(Document(page_content=r["_source"]["answer"]))
        return docs


bm25_retriever = OpenSearchBM25Retriever(client=client, index_name=index_name)

bm25_retriever.get_relevant_documents(question)


[Document(page_content='Q: Amazon EC2 で何ができますか?\nAmazon Simple Storage Service (Amazon S3) がクラウド内のストレージを可能とするのとまったく同様に、Amazon EC2 は、クラウド内での「コンピューティング」を可能にします。\xa0 Amazon EC2 のシンプルなウェブサービスインターフェイスによって、手間をかけず、必要な機能を取得および設定できます。お客様のコンピューティングリソースに対して、高機能なコントロールが提供され、Amazon の実績あるインフラストラクチャ上で実行できます。Amazon EC2 では、わずか数分間で新規サーバーインスタンスを取得して起動できるようになります。これにより、コンピューティング要件の変化に合わせて、すばやく容量をスケールアップおよびスケールダウンできます。実際に使用した分のみ料金が発生するため、Amazon EC2 はコンピューティングの経済性も変革します。'),
 Document(page_content='Q:Nitro Hypervisor では、インスタンスの再起動と終了の EC2 API リクエストがどのように実装されますか?\nNitro Hypervisor は、インスタンスで実行されているオペレーティングシステムに、業界標準の ACPI メソッドによって正常にシャットダウンする必要があることを通知します。ただし、Linux インスタンスの場合は、acpid がインストールされており、正常に機能している必要があります。インスタンスで acpid が機能していない場合、終了イベントは数分間遅れ、ハードリセットまたは電源オフとして実行されます。'),
 Document(page_content='Q:「EC2 コンピューティングユニット (ECU)」とは何ですか? またそれを導入する理由は何ですか?\nユーティリティコンピューティングモデルへの移行は、デベロッパーが CPU リソースに対してこれまで持っていた考え方を根本的に変えるものです。特定のプロセッサを購入またはリースして数か月から数年間使用する代わりに、時間単位で処理能力をレンタルすることができます。Amazon EC2 は、コモディティなハードウェア上に構築さ

In [42]:
from langchain.chains import ConversationalRetrievalChain

bm25_qa = ConversationalRetrievalChain.from_llm(
  llm, 
  retriever=bm25_retriever
  )

bm25_response = bm25_qa({"question": question, "chat_history": [] })


In [33]:
print(bm25_response['answer'])

 Amazon EC2では、以下のような主なOSが起動できます。

- Linux固定AMI(Amazon Linux、Ubuntu、CentOSなど)
- Windowsサーバー
- FreeBSD
- Oracle Linux
- SUSE Linux
- Red Hat Enterprise Linux (RHEL) 
- macOS
- Other Linux distributions

EC2では、これらの主要OSに加えて、ユーザが独自に作成したカスタムAMIからの起動も可能です。AMI(Amazon Machine Image)は仮想マシンのスナップショットで、任意のOSとソフトウェアをプレロードできるため、ユーザの要件に合わせたカスタムOS/ソフトウェア構成のEC2インスタンスを起動できます。


---
## 2. ニューラル検索


In [23]:
from typing import Any, Iterable, List

from langchain.callbacks.manager import CallbackManagerForRetrieverRun
from langchain.docstore.document import Document
from langchain.schema import BaseRetriever

class OpenSearchNeuralRetriever(BaseRetriever):

    client: Any
    index_name: str
    model_id: str

    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:

        body = {
            "_source": {
                "excludes": [
                    "passage_embedding"
                ]
            },
            "query": {
                "neural": {
                    embedding_field: {
                        "query_text": question,
                        "model_id": self.model_id,
                    }
                }
            }
        }

        res  = self.client.search(index=index_name,body=body)

        docs = []
        for r in res["hits"]["hits"]:
            docs.append(Document(page_content=r["_source"]["answer"]))
        return docs


neural_retriever = OpenSearchNeuralRetriever(client=client, index_name=index_name, model_id=model_id)

neural_retriever.get_relevant_documents(question)


[Document(page_content='Q: どうすれば Amazon EC2 の使用を開始できますか?\nAmazon EC2 にサインアップするには、Amazon EC2 詳細ページ上の [このウェブサービスにサインアップ] ボタンを選択します。このサービスにアクセスするには、AWS のアカウントを保有している必要があります。これをまだ持っていない場合は、Amazon EC2 サインアッププロセスの開始時に、プロンプト画面が表示されてこれを作成することができます。サインアップの後、Amazon EC2 ドキュメントをご参照ください。ここには開始方法に関するガイドが含まれています。'),
 Document(page_content='Q: 既存の Windows Server ライセンスを EC2 で使用できますか?\nはい、できます。ImportImage ツールを使用して自分の Windows Server マシンイメージをインポートした後、それらのマシンイメージから EC2 Dedicated Hosts にインスタンスを起動して、インスタンスを効果的に管理し、使用量をレポートすることができます。通常、Microsoft はソケットやコアなどの物理リソースに応じてライセンス使用量を追跡するように要求しますが、Dedicated Hosts を使用するとそれが簡単に行えます。自分の Windows Server ライセンスを Amazon EC2 Dedicated Hosts で使用する方法について詳しくは Dedicated Hosts の詳細に関するページを参照してください。'),
 Document(page_content='Q: EC2 Mac インスタンスで EC2 ユーザーデータを使用するにはどうすればよいですか?\nEC2 Linux および Windows インスタンスと同様に、EC2 Mac インスタンスにカスタムユーザーデータを渡すことができます。EC2 Mac インスタンスは、cloud-init の代わりに、オープンソースの起動デーモンである ec2-macos-init を使用します。お客様はこのデータを EC2 Launch Wizard に、プレーンテキスト、ファイル、または Base64 エンコードテキストとして渡

In [43]:
from langchain.chains import ConversationalRetrievalChain

neural_qa = ConversationalRetrievalChain.from_llm(
  llm, 
  retriever=neural_retriever
  )

neural_response = neural_qa({"question": question, "chat_history": [] })


In [44]:
print(neural_response['answer'])


 Amazon EC2で起動できる主なOSは以下のとおりです。

- Linux各ディストリビューション(Amazon Linux、Ubuntu、CentOSなど)
- Windows Server(Windows Server 2022など)
- VMware ESXi
- Oracle Linux 
- SUSE Linux
- Red Hat Enterprise Linux
- FreeBSD
- Oracle Solaris
- Apache Mavericks

EC2上で新しいOSイメージ(AMI)を作成することも可能で、独自の環境やアプリケーションを配備できます。

主な技術的な制約は以下の通りです。

- 64ビットアーキテクチャのOSのみサポート
- グラフィカルなデスクトップ環境はサポートされない
- 特定のEC2インスタンスタイプではサポートされないOSがある

具体的なEC2でサポートされているAMIのリストはAWSのサイト内のドキュメントを確認する必要があります。如何なるOSでも実行できる訳ではありませんが、一般的なLinuxディストリビューションやWindowsサーバオペレーティングシステムは多くのEC2インスタンスタイプで実行可能です。


---
## 3. ハイブリッド検索


In [49]:
from typing import Any, Iterable, List

from langchain.callbacks.manager import CallbackManagerForRetrieverRun
from langchain.docstore.document import Document
from langchain.schema import BaseRetriever

class OpenSearchHybridRetriever(BaseRetriever):

    client: Any
    index_name: str
    model_id: str

    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:

        body = {
            "_source": {
                "excludes": [
                    "passage_embedding"
                ]
            },
            "query": {
                "hybrid": {
                    "queries": [
                        {
                            "match": {
                                "question": {
                                    "query": question
                                }
                            }
                        },
                        {
                            "neural": {
                                "passage_embedding": {
                                    "query_text": question,
                                    "model_id": model_id,
                                }
                            }
                        }
                    ]
                }
            }
        }

        res  = self.client.search(index=index_name,body=body)

        docs = []
        for r in res["hits"]["hits"]:
            docs.append(Document(page_content=r["_source"]["answer"]))
        return docs


hybrid_retriever = OpenSearchHybridRetriever(client=client, index_name=index_name, model_id=model_id)

hybrid_retriever.get_relevant_documents(question)


[Document(page_content='Q: Amazon EC2 で何ができますか?\nAmazon Simple Storage Service (Amazon S3) がクラウド内のストレージを可能とするのとまったく同様に、Amazon EC2 は、クラウド内での「コンピューティング」を可能にします。\xa0 Amazon EC2 のシンプルなウェブサービスインターフェイスによって、手間をかけず、必要な機能を取得および設定できます。お客様のコンピューティングリソースに対して、高機能なコントロールが提供され、Amazon の実績あるインフラストラクチャ上で実行できます。Amazon EC2 では、わずか数分間で新規サーバーインスタンスを取得して起動できるようになります。これにより、コンピューティング要件の変化に合わせて、すばやく容量をスケールアップおよびスケールダウンできます。実際に使用した分のみ料金が発生するため、Amazon EC2 はコンピューティングの経済性も変革します。'),
 Document(page_content='Q: Amazon EC2 で何ができますか?\nAmazon Simple Storage Service (Amazon S3) がクラウド内のストレージを可能とするのとまったく同様に、Amazon EC2 は、クラウド内での「コンピューティング」を可能にします。\xa0 Amazon EC2 のシンプルなウェブサービスインターフェイスによって、手間をかけず、必要な機能を取得および設定できます。お客様のコンピューティングリソースに対して、高機能なコントロールが提供され、Amazon の実績あるインフラストラクチャ上で実行できます。Amazon EC2 では、わずか数分間で新規サーバーインスタンスを取得して起動できるようになります。これにより、コンピューティング要件の変化に合わせて、すばやく容量をスケールアップおよびスケールダウンできます。実際に使用した分のみ料金が発生するため、Amazon EC2 はコンピューティングの経済性も変革します。'),
 Document(page_content='Q: Amazon EC2 で何ができますか?\nAmazon Simple Storage Service (Amazon S

In [50]:
from langchain.chains import ConversationalRetrievalChain

hybrid_qa = ConversationalRetrievalChain.from_llm(
  llm, 
  retriever=hybrid_retriever
  )

hybrid_response = hybrid_qa({"question": question, "chat_history": [] })


In [51]:
print(hybrid_response['answer'])


 Amazon EC2では、主要なオペレーティングシステムがサポートされています。

- Linuxおよびそのディストリビューション(Amazon Linux、Ubuntu、CentOSなど)

- Windows Server

- FreeBSD

- Oracle Linux

- Redis Enterprise Flash Storage

- SUSE Linux

- VMware ESXi

- Citrix XenServer

以上がEC2で起動可能な主要なオペレーティングシステムです。Linuxディストリビューションが最も幅広くサポートされており、WindowsServerや一部のUnix系OSもサポートされています。EC2では様々なOSから選択できるため、お客様の必要なOS環境でサービスを拡張できます。


---
いろいろ試す
---

In [52]:
questions = [
  "EC2でMacOSは起動できますか？",
  "EC2の優れた点を教えてください。",
  "EC2とコンテナの違いを教えて"
]

for q in questions:
  print("=====")
  print(q)
  print("-----BM25-----")
  res = bm25_qa({"question": q, "chat_history": [] })
  print(res['answer'])
  print("-----")
  print("-----Neural-----")
  res = neural_qa({"question": q, "chat_history": [] })
  print(res['answer'])
  print("-----")
  print("-----Hybrid-----")
  res = hybrid_qa({"question": q, "chat_history": [] })
  print(res['answer'])
  print("-----")
  print("-----")


=====
EC2でMacOSは起動できますか？
-----BM25-----
 はい、EC2(Amazon Elastic Compute Cloud)ではMacOSを実行できるインスタンスタイプが用意されています。

EC2にはEC2 Macインスタンスという、Mac miniハードウェアをベースとしたインスタンスタイプがあります。このインスタンスタイプではMacOSが起動・実行可能です。

現在提供されているEC2 Macインスタンスは主に2つあります:

- x86アーキテクチャベースのEC2 Macインスタンス: Mac mini(Late 2018)がベースで、MacOS Mojaveが起動できます。

- M1アーキテクチャベースのEC2 Macインスタンス: Mac mini(2020)がベースで、MacOS Big Surが起動できます。

このようにEC2では物理的なMacハードウェアを利用したEC2 Macインスタンスを通じて、MacOSの実行がサポートされています。
-----
-----Neural-----
 はい、Amazon EC2ではMacOSの仮想マシン(VM)を起動できます。

Amazon EC2 for Macについての主な点は以下のとおりです:

- EC2 for Macは、Mac mini仮想マシンとして提供されています。MacOS High Sierra(10.13)やMacOS Mojave(10.14)がサポートされています。 

- EC2 for Macインスタンスはオンデマンドおよびスポットインスタンスとして利用可能です。

- CPU、メモリ、ストレージの構成はEC2の他のインスタンスタイプと同様にカスタマイズ可能です。

- MacOS VMは専用か共有のサーバーホスト上で実行され、専用ホストの場合は他の顧客インスタンスとの影響を受けません。

- 安定性、セキュリティ、スケーラビリティがAWSのインフラによってサポートされます。

- EC2 for Macは開発、テスト、デプロイといった用途で使用されています。

以上の通り、EC2ではMacOS VMを簡単に起動し、クラウドのメリッツをMacOS環境でも利用できるようになっています。
-----
-----Hybrid-----
 はい、Amazo